# PROBLEM STATEMENT
A US bike-sharing provider BoomBikes has recently suffered considerable dips 
in their revenues due to the ongoing Corona pandemic. The company is finding 
it very difficult to sustain in the current market scenario. So, it has decided 
to come up with a mindful business plan to be able to accelerate its revenue as 
soon as the ongoing lockdown comes to an end, and the economy restores to a healthy state. 

# Business Goal

We are required to model the demand for shared bikes with the available independent variables. It will be used by the management to understand how exactly the demands vary with different features. They can accordingly manipulate the business strategy to meet the demand levels and meet the customer's expectations. Further, the model will be a good way for management to understand the demand dynamics of a new market.

In [ ]:
# Supress Warnings and importing essential Libs

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [ ]:
# Reading_Data
df = pd.read_csv("day.csv")

In [ ]:
# Check the head of the dataset
df.head()

Description of following fields:
	
	- instant: record index
	- dteday : date
	- season : season (1:spring, 2:summer, 3:fall, 4:winter)
	- yr : year (0: 2018, 1:2019)
	- mnth : month ( 1 to 12)
	- holiday : weather day is a holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit : 
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : temperature in Celsius
	- atemp: feeling temperature in Celsius
	- hum: humidity
	- windspeed: wind speed
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

## Step 2: Visualising the Data

If there is some obvious multicollinearity going on, this is the first place to catch it
Here's where you'll also identify if some predictors directly have a strong association with the outcome variable
We'll visualise our data using matplotlib and seaborn.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.pairplot(df)
plt.show()

# OBSERVATIONS:

There are no null values present

We have both numerical and categorical values

instant column is index which does not have any significance in our analysis. So we'll drop the column

The varibles casual and registered are summed up to get cnt which is our target variable. 
Also during prediction we will not be having these data, so we will drop these two variables 
which we are not going to use in the model.

We don't find any peculiar outliers after analysing description of dataframe

'cnt' /demand of shared bikes definitely has some sort of relationship with 'temp'/ 'apparent_temp'.

We are going to use weekday varible which is derived from dteday, so we will not be using dteday and will drop it.

temp and atemp are directly correlated among each other. We will use temp and drop atemp.

In [ ]:
# drop observed unnecesary columns instance, dteday, casual, registered and atemp
df.drop(['instant', 'dteday','casual','registered','atemp'], axis=1, inplace=True)

# CATEGORICAL ANALYSIS

In [ ]:
# look at the correaltion between continous varibales using heat map
plt.figure(figsize=(20,10))
sns.heatmap(df.corr(), annot=True)
plt.show()

OBSERVATIONS:

    A positive correalation observed between cnt and temp (0.63)
    A Negative correlation observed for cnt with hum,holiday and windspeed (-0.099,-0.069 and -0.24)

In [ ]:
# BoxPlot for categorical variables
col = 3
categorical_vars = ['season','yr','mnth','holiday','weekday','workingday','weathersit']
row = len(categorical_vars)//col+1

plt.figure(figsize=(15,12))
for i in list(enumerate(categorical_vars)):
    plt.subplot(row,col,i[0]+1)
    sns.boxplot(x = i[1], y = 'cnt', data = df)
    plt.xticks(rotation = 90)
plt.tight_layout(pad = 1)    
plt.show()

# DATA CREATION/DUMMY VARIABLES INFER

In [ ]:
# Following categorical variables are mapped according to Data Dictionary :season, year, month, weekday, weathersit

df['season']=df['season'].map({1: 'spring', 2: 'summer',3:'fall', 4:'winter'})
df['yr']=df['yr'].map({0: '2018', 1: '2019'})
df['mnth']=df['mnth'].map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'June', 7:'July', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'})
df['weekday']=df['weekday'].map({0:'Sun', 1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu', 5:'Fri', 6:'Sat'})
df['weathersit']=df['weathersit'].map({1: 'Clear/Partly Cloudy', 2:'Mist/Cloudy', 3:'Light Snow/Rain', 4:'Heavy Snow/Rain/Hail/Fog'})

df.head()

In [ ]:
# get dummy variables for season, weekday, mnth and weathersit
dummy_vars = pd.get_dummies(df[['season','weekday','mnth','weathersit','yr']],drop_first=True)

# concat the dummy df with original df
df = pd.concat([df,dummy_vars], axis = 1)

# drop season column
df.drop(['season','weekday','mnth','weathersit','yr'], axis=1, inplace=True)

df.head()

In [ ]:
df.info()

In [ ]:
# Convert categorical columns to numeric 
df[['holiday','workingday']]= df[['holiday','workingday']].astype('uint8')
df.info()

# DATA SPLIT

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score, mean_squared_error

import statsmodels.api as sm

In [ ]:
# Split train-test dataset
df_train, df_test = train_test_split(df, train_size = 0.7, random_state = 100 )
print(df_train.shape)
print(df_test.shape)

Min-Max Scaler

In [ ]:
# Scaling of train set

scaler = MinMaxScaler()

# fit and transform on training data
df_train[['temp', 'hum','windspeed','cnt']] = scaler.fit_transform(df_train[['temp', 'hum','windspeed','cnt']])
df_train.head()

In [ ]:
# Only transform test dataset 
df_test[['temp', 'hum','windspeed','cnt']] = scaler.transform(df_test[['temp', 'hum','windspeed','cnt']])
df_test.head()

SPLIT DATASET 

In [ ]:
# Creating X and y data dataframe for train set
y_train = df_train.pop('cnt')
X_train = df_train
X_train.head()

In [ ]:
# Creating X and y data dataframe for test set
y_test = df_test.pop('cnt')
X_test = df_test
X_test.head()

DATA MODELLING USING RFE AND MANUAL SELECTION

In [ ]:
# Running RFE to select 15 number of varibleslm = LinearRegression()
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm,  n_features_to_select=15)
rfe = rfe.fit(X_train, y_train)

In [ ]:
col = X_train.columns[rfe.support_]
col

In [ ]:
#BUILDING USING RFE COLUMNS

X_train_rfe = X_train[col]

In [ ]:
# create function for stats linear model 
def sm_linearmodel(X_train_sm):
    #Add constant
    X_train_sm = sm.add_constant(X_train_sm)

    # create a fitted model (1st model)
    lm = sm.OLS(y_train,X_train_sm).fit()
    return lm

In [ ]:
# Function to calculate VIF
# calculate VIF
def vif_calc(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'],2)
    vif = vif.sort_values(by='VIF', ascending = False)
    return vif

In [ ]:
# Create 1st stats model and look for summary and VIF
lm_1 = sm_linearmodel(X_train_rfe)
print(lm_1.summary())

# Calculate VIF
print(vif_calc(X_train_rfe))

In [ ]:
# Loop to remove P value variables >0.05 in bstep mannen and update model

pvalue = lm_1.pvalues
while(max(pvalue)>0.05):
    maxp_var = pvalue[pvalue == pvalue.max()].index
    print('Removed variable:' , maxp_var[0], '    P value: ', round(max(pvalue),3))
    
    # drop variable with high p value
    X_train_rfe = X_train_rfe.drop(maxp_var, axis = 1)
    lm_1 = sm_linearmodel(X_train_rfe)
    pvalue = lm_1.pvalues

In [ ]:
print(lm_1.summary())

print(vif_calc(X_train_rfe))

In [ ]:
# drop varible having high VIF
X_train_new = X_train_rfe.drop(['hum'],axis = 1)

# Create stats model and look for summary
lm_2 = sm_linearmodel(X_train_new)
print(lm_2.summary())

# Calculate VIF
print(vif_calc(X_train_new))

In [ ]:
# drop varible having high VIF
X_train_new = X_train_new.drop(['workingday'],axis = 1)

# Create stats model and look for summary
lm_2 = sm_linearmodel(X_train_new)
print(lm_2.summary())

# Calculate VIF
print(vif_calc(X_train_new))

In [ ]:
# drop varible having high VIF
X_train_new = X_train_new.drop(['temp'],axis = 1)

# Create stats model and look for summary
lm_3 = sm_linearmodel(X_train_new)
print(lm_3.summary())

# Calculate VIF
print(vif_calc(X_train_new))

In [ ]:
# drop varible having high VIF
X_train_new = X_train_new.drop(['weekday_Sat'],axis = 1)

# Create stats model and look for summary
lm_4 = sm_linearmodel(X_train_new)
print(lm_4.summary())

# Calculate VIF
print(vif_calc(X_train_new))

In [ ]:
# drop varible having high VIF
X_train_new = X_train_new.drop(['mnth_July'],axis = 1)

# Create stats model and look for summary
lm_5 = sm_linearmodel(X_train_new)
print(lm_5.summary())

# Calculate VIF
print(vif_calc(X_train_new))

In [ ]:
# # drop varible having high VIF
# X_train_new = X_train_new.drop(['hum'],axis = 1)

# # Create stats model and look for summary
# lm_6 = sm_linearmodel(X_train_new)
# print(lm_6.summary())

# # Calculate VIF
# print(vif_calc(X_train_new))

In [ ]:
# List down final model varibales and its coefficients

# assign final model to lm_final
lm_final = lm_5

# list down and check variables of final model
var_final = list(lm_final.params.index)
var_final.remove('const')
print('Final Selected Variables:', var_final)

# Print the coefficents of final varible
print('\033[1m{:10s}\033[0m'.format('\nCoefficent for the variables are:'))
print(round(lm_final.params,3))

RESIDUAL ANALYSIS

In [ ]:
# Select final variables from the test dataset
X_train_res = X_train[var_final]

In [ ]:
#Add constant
X_train_res = sm.add_constant(X_train_res)

#Predict train set
y_train_pred = lm_final.predict(X_train_res)

In [ ]:
# distrubition plot for residue
res = y_train - y_train_pred
sns.distplot(res)
plt.title('Error terms')
plt.show()

In [ ]:
# Error terms train set
c = [i for i in range(1,len(y_train)+1,1)]
fig = plt.figure(figsize=(8,5))
plt.scatter(y_train,res)
fig.suptitle('Error Terms', fontsize=16)            # Plot heading 
plt.xlabel('Y_train_pred', fontsize=14)             # X-label
plt.ylabel('Residual', fontsize=14) 

PREDICTIONS

In [ ]:
# select final variables from X_test
X_test_sm = X_test[var_final]
X_test_sm.head()

In [ ]:
# add constant
X_test_sm = sm.add_constant(X_test_sm)
X_test_sm.head()

In [ ]:
# predict test dataset
y_test_pred = lm_final.predict(X_test_sm)

EVALUATION

In [ ]:
# Get R-Squared fro test dataset
r2_test = r2_score(y_true = y_test, y_pred = y_test_pred)
print('R-Squared for Test dataset: ', round(r2_test,3))

In [ ]:
# Adj. R-Squared for test dataset
N= len(X_test)          # sample size
p =len(var_final)     # Number of independent variable
r2_test_adj = round((1-((1-r2_test)*(N-1)/(N-p-1))),3)
print('Adj. R-Squared for Test dataset: ', round(r2_test_adj,3))

In [ ]:
res_test = y_test - y_test_pred
plt.title('Error Terms', fontsize=16) 
sns.distplot(res_test)
plt.show()

## QnA's

1. From your analysis of the categorical variables from the dataset, what could you infer about 
their effect on the dependent variable?

CRITICAL OBSERVATIONS

The demad of bike is less in the season 1 ie spring
The demand bike increased in the year 2019 wrt year 2018.
Month Jun - Sep is the period when bike demand is high. The Months Jan-Feb have lowest demand.
Bike demand is less in holidays .


2. Why is it important to use drop_first=True during dummy variable creation? 

drop_first=True is important to use, as it helps in reducing the extra column created during dummy variable creation. Hence it reduces the correlations created among dummy variables.

3. Looking at the pair-plot among the numerical variables, which one has the highest correlation 
with the target variable?

'temp' and 'atemp'


4. How did you validate the assumptions of Linear Regression after building the model on the 
training set? 

Assumption of Regression Model : 
•	Linearity: The relationship between dependent and independent variables should be linear.

•	Homoscedasticity: Constant variance of the errors should be maintained.

•	Multivariate normality: Multiple Regression assumes that the residuals are normally distributed.

•	Lack of Multicollinearity: It is assumed that there is little or no multicollinearity in the data.



5. Based on the final model, which are the top 3 features contributing significantly towards 
explaining the demand of the shared bikes?

Top picks would be:

    'temp'
    
    'yr'
    
    'weathersit'

1. Explain the linear regression algorithm in detail.

Linear Regression is a machine learning algorithm based on supervised learning. It performs a regression task. Regression models a target prediction value based on independent variables. It is mostly used for finding out the relationship between variables and forecasting. Different regression models differ based on – the kind of relationship between dependent and independent variables they are considering, and the number of independent variables getting used.
Linear regression performs the task to predict a dependent variable value (y) based on a given independent variable (x). So, this regression technique finds out a linear relationship between x (input) and y(output). Hence, the name is Linear Regression.
In the figure above, X (input) is the work experience and Y (output) is the salary of a person. The regression line is the best fit line for our model.


2. Explain the Anscombe’s quartet in detail. 

Anscombe’s quartet tells us about the importance of visualizing data before applying various algorithms to build models. This suggests the data features must be plotted to see the distribution of the samples that can help you identify the various anomalies present in the data (outliers, diversity of the data, linear separability of the data, etc.). Moreover, the linear regression can only be considered a fit for the data with linear relationships and is incapable of handling any other kind of data set. 

3. What is Pearson’s R? 


In statistics, the Pearson correlation coefficient (PCC), also referred to as Pearson's r, the Pearson product-moment correlation coefficient (PPMCC), or the bivariate correlation, is a measure of linear correlation between two sets of data. It is the covariance of two variables, divided by the product of their standard deviations; thus it is essentially a normalised measurement of the covariance, such that the result always has a value between −1 and 1.

4. What is scaling? Why is scaling performed? What is the difference between normalized scaling 
and standardized scaling? 

It is a step of data Pre-Processing which is applied to independent variables to normalize the data within a particular range. It also helps in speeding up the calculations in an algorithm.

Most of the times, collected data set contains features highly varying in magnitudes, units and range. If scaling is not done then algorithm only takes magnitude in account and not units hence incorrect modelling. To solve this issue, we have to do scaling to bring all the variables to the same level of magnitude.

It is important to note that scaling just affects the coefficients and none of the other parameters like t-statistic, F-statistic, p-values, R-squared, etc.

Normalization/Min-Max Scaling:
It brings all of the data in the range of 0 and 1. sklearn.preprocessing.MinMaxScaler helps to implement normalization in python.

Standardization Scaling:
Standardization replaces the values by their Z scores. It brings all of the data into a standard normal distribution which has mean (μ) zero and standard deviation one (σ).

sklearn.preprocessing.scale helps to implement standardization in python.
One disadvantage of normalization over standardization is that it loses some information in the data, especially about outliers.



5. You might have observed that sometimes the value of VIF is infinite. Why does this happen?

An infinite value of VIF for a given independent variable indicates that it can be perfectly predicted by other variables in the model.



6. What is a Q-Q plot? Explain the use and importance of a Q-Q plot in linear regression.

Quantile-Quantile (Q-Q) plot, is a graphical tool to help us assess if a set of data plausibly came from some theoretical distribution such as a Normal, exponential or Uniform distribution. Also, it helps to determine if two data sets come from populations with a common distribution.

This helps in a scenario of linear regression when we have training and test data set received separately and then we can confirm using Q-Q plot that both the data sets are from populations with same distributions.

Few advantages:
a) It can be used with sample sizes also

b) Many distributional aspects like shifts in location, shifts in scale, changes in symmetry, and the presence of outliers can all be detected from this plot.